## Code

In [1]:
%load_ext ipython_sparql_pandas

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
from random import randint
sparql = SPARQLWrapper('http://MacBook-Air-van-Daphne.local:7200/repositories/statements')
sparql.setReturnFormat(JSON)

In [3]:
#from the query results extract a list that contains the counts after splitting on certain attributes
PosAttr=['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
NegAttr=[]
AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']

def extractCountAndAttr(result):
    number = []
    listOfObj = []
    listOfPred = []
    listOfCompQ = []
    for attr in result["results"]["bindings"]:
        listOfObj.append(attr['o'])
        listOfPred.append(attr['p'])
        listOfCompQ.append(attr['isCompound'])
    return(listOfObj,listOfPred,listOfCompQ)

def generateQuestion(listOfPred, listOfObj, listOfCompQ, index, subject):
    flag = True
    
    if listOfCompQ[index]['value'] == 'false':  
        if ('<' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>') in AttrHistory:
            return generateQuestion(listOfPred, listOfObj, listOfCompQ, index+1, subject)

        else:
            AttrHistory.append('<' + str(listOfPred[index]['value']) +'> <'+str(listOfObj[index]['value'])+ '>')
    
    if listOfCompQ[index]['value'] == 'true':  
        if ('<' + str(listOfPred[index]) +'> [<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <'+str(listOfObj[index])+ '>]') in AttrHistory:
            return generateQuestion(listOfPred, listOfObj, listOfCompQ, index+1, subject)

        else:
            AttrHistory.append('<' + str(listOfPred[index]['value']) +'> [<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <'+ str(listOfObj[index]['value']) + '>]')
    
    try:
        int(listOfObj[index]['value'][0])
    except ValueError:
           flag = False
    
    #if listOfCompQ[index]['value'] == 'false':    
    #    print(f'Does the thing you are looking for have the attribute: {listOfPred[index]["value"].split("/")[-1]} {listOfObj[index]["value"].split("/")[-1]}?' )
    
    #elif listOfCompQ[index]['value'] == 'true':   
    #    print(f'Does the thing you are looking for have the attribute: {listOfPred[index]["value"].split("/")[-1]} of type {listOfObj[index]["value"].split("/")[-1]}?' )
    
    if flag is True:
        datatype = listOfObj[index]["datatype"].split("#")[-1]
        answer = findAnswer(subject, '<' + str(listOfPred[index]["value"]) +'>', '"'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype, listOfCompQ[index]['value'])
    else:
        answer = findAnswer(subject, '<' + str(listOfPred[index]["value"]) +'>', '<'+str(listOfObj[index]["value"])+ '>', listOfCompQ[index]['value'])
    #print(answer)
    
    if answer:   
        if flag is True:
            datatype = listOfObj[index]["datatype"].split("#")[-1]
            
            if listOfCompQ[index]['value'] == 'false':
                PosAttr.append('?s <' + str(listOfPred[index]["value"]) + '> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'.') 
            elif listOfCompQ[index]['value'] == 'true':
                PosAttr.append('?s <' + str(listOfPred[index]["value"]) + '> [<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> "' + str(listOfObj[index]["value"]) + '"^^xsd:'+datatype+'].') 
        
        else:
            if listOfCompQ[index]['value'] == 'false':
                PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> <'+str(listOfObj[index]["value"])+ '>.')
            elif listOfCompQ[index]['value'] == 'true':
                PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> [<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <'+str(listOfObj[index]["value"])+ '>].')
        
        
    elif not answer:
        if flag is True:
            datatype = listOfObj[index]["datatype"].split("#")[-1]
            
            if listOfCompQ[index]['value'] == 'false':
                NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'. }}')
            elif listOfCompQ[index]['value'] == 'true':
                NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> [<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+']. }}')
        
        else:
            if listOfCompQ[index]['value'] == 'false':
                NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> <'+str(listOfObj[index]["value"])+ '>. }}')   
            elif listOfCompQ[index]['value'] == 'true':
                NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> [<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <'+str(listOfObj[index]["value"])+ '>]. }}')   
                
    NegFilters = "\n".join(NegAttr)
    PosFilters = "\n".join(PosAttr)
    return (PosFilters, NegFilters)

In [4]:
 def updateQuery(left, PosFilters, NegFilters):
    FilterQuestion =  (f"""
        SELECT (count(distinct ?s) as ?count) ?p ?o ?isCompound WHERE {{
        {PosFilters}
        {{?s ?p [<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> ?o] . BIND (true as ?isCompound) }} UNION {{?s ?p ?o . BIND (false as ?isCompound) }}
        {NegFilters}
        filter (?o != <http://www.w3.org/2002/07/owl#Class>)
        filter (?o != <http://example.com/popularEntity> )
        }}
        group by ?p ?o ?isCompound
        order by abs({left} - ?count )
        limit 5            
        """) 
    return FilterQuestion

In [5]:
def pickSample():
    query = f"""
    select distinct ?s
    where {{
        ?s ?p ?o .
        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
    }}
    offset 5678
    limit 1000
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    return qres["results"]["bindings"]

In [6]:
def findAnswer(subject,p,o,compq):
    query = f"""
        SELECT ?s ?p ?o
        WHERE {{
        {{?s ?p ?o1 . }}
        UNION
        {{?s ?p [<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> ?o2] .}}
        FILTER(?s = {subject})
        FILTER(?p = {p})
        FILTER(if({compq}, ?o2, ?o1) = {o})
        }}"""
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    res = qres["results"]["bindings"]
    return (len(res))>0

In [7]:
def numberleft(PosFilters,NegFilters):
    query =  f"""
            select (count(distinct ?s) as ?count) where 
            {{
            {PosFilters}            
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            {NegFilters} 
            }}       
            group by ?p ?o 
            ORDER BY DESC(?count )            
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    for r in qres["results"]["bindings"]:
        left= (r["count"]["value"])
        return(int(left))

In [8]:
def popentities(PosFilters,NegFilters):
    query =  f"""
            select ?s where 
            {{
            
            {PosFilters}
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            {NegFilters} 
            
            }}
            
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    return qres["results"]["bindings"]

In [9]:
# resets all variables
def game_reset():
    PosAttr= ['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .']
    NegAttr= ['']
    AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']
    left = numberleft("".join(PosAttr), "".join(NegAttr))
    FilterQuestion =  f"""
        SELECT (count(distinct ?s) as ?count) ?p ?o ?isCompound WHERE {{
        {{?s ?p [<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> ?o] . BIND (true as ?isCompound) }} UNION {{?s ?p ?o . BIND (false as ?isCompound) }}
        ?s a <http://example.com/popularEntity> .
        filter (?o != <http://www.w3.org/2002/07/owl#Class>)
        filter (?o != <http://example.com/popularEntity> )
        }}
        group by ?p ?o ?isCompound
        order by abs( %s - ?count )
        limit 5
        """ %((left/2))
    sparql.setQuery(FilterQuestion)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    listOfObj ,listOfPred, listOfCompQ = extractCountAndAttr(qres)
    return (left, qres, PosAttr, NegAttr, AttrHistory)

In [10]:
# without printing results
import random

wincount = 611
gamesplayed = 857
totalgames = 1000
sum_avg_questions = 8509
avg_questions = sum_avg_questions / wincount

sample = []
sample_dict = pickSample()
for subject in sample_dict:
    sample.append(subject["s"]["value"])

while gamesplayed < totalgames:
    # Resets all variables
    left,qres,PosAttr,NegAttr,AttrHistory = game_reset()
    i=0
    outofguess = []
    
    # pick subject
    subject = ("<" + str(sample[gamesplayed]) + ">")

    while i < 20:

        # list of Pred and Obj for questions
        listOfObj ,listOfPred, listOfCompQ = extractCountAndAttr(qres)

        # if only 1 popular entity is left break
        if left<=1:
            break

        # ask question and add filters to query
        try: 
            PosFilters, NegFilters = generateQuestion(listOfPred, listOfObj, listOfCompQ, 0, subject)
            exception_occured = False
        # If time-out limit reached
        except: 
            exception_occured = True
            gamesplayed += 1
            print("Query took too long\nCorrect: " + str(wincount) + "/" + str(gamesplayed) + " (average: " + str(avg_questions) + " questions)")
            break

        # number of popular entities left
        left = numberleft(PosFilters, NegFilters)

        # run query
        query = updateQuery(left/2,PosFilters, NegFilters)
        sparql.setQuery(query)
        qres = sparql.queryAndConvert()

        i+=1
        
    if exception_occured:
        continue

    # Prints answer
    if i != 20:
        for attr in popentities(PosFilters, NegFilters):
            for x in [attr['s']['value']][0:10]:
                answer = (x.split("/")[-1])
        if str(answer.split("/")[-1] + ">") == str(subject.split("/")[-1]):
            wincount += 1
            gamesplayed += 1
            sum_avg_questions += i
            avg_questions = sum_avg_questions/wincount
        else:
            gamesplayed += 1
        print("Correct: " + str(wincount) + "/" + str(gamesplayed) + " (average: " + str(avg_questions) + " questions)")
    # ran out of questions
    else:
        for j in popentities(PosFilters, NegFilters):
                outofguess.append(f'{j["s"]["value"].split("/")[-1]}')
        guess = random.choice(outofguess)
        if str(subject.split("/")[-1]) == str(guess + ">"):
            wincount += 1
            gamesplayed += 1
            sum_avg_questions += i
            avg_questions = sum_avg_questions/wincount
        else:
            gamesplayed += 1
        print("Correct: " + str(wincount) + "/" + str(gamesplayed) + " (average: " + str(avg_questions) + " questions)")
            
print("\nTournament ended! I have guessed " + str(wincount) + " out of " + str(gamesplayed) + " things correctly (average: " + str(avg_questions) + " questions).")

Query took too long
Correct: 611/858 (average: 13.926350245499181 questions)
Correct: 611/859 (average: 13.926350245499181 questions)
Correct: 611/860 (average: 13.926350245499181 questions)
Correct: 611/861 (average: 13.926350245499181 questions)
Correct: 612/862 (average: 13.931372549019608 questions)
Correct: 613/863 (average: 13.931484502446983 questions)
Correct: 614/864 (average: 13.938110749185668 questions)
Correct: 615/865 (average: 13.93658536585366 questions)
Correct: 616/866 (average: 13.936688311688311 questions)
Correct: 617/867 (average: 13.93679092382496 questions)
Query took too long
Correct: 617/868 (average: 13.93679092382496 questions)
Query took too long
Correct: 617/869 (average: 13.93679092382496 questions)
Correct: 618/870 (average: 13.936893203883495 questions)
Correct: 619/871 (average: 13.938610662358643 questions)
Correct: 620/872 (average: 13.94032258064516 questions)
Correct: 621/873 (average: 13.938808373590982 questions)
Correct: 622/874 (average: 13.937